In [2]:
import numpy as np
import os

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
from keras.layers import Input, Dense, Flatten, Convolution2D, BatchNormalization, Activation, Add
from keras.models import Model
from keras.utils import to_categorical
from keras import backend as K
from pommerman.constants import BOARD_SIZE
from tqdm import tqdm_notebook as tqdm
from sklearn.utils.class_weight import compute_class_weight

/gpfs/hpchome/anton95/.conda/envs/pommerman/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/gpfs/hpchome/anton95/.conda/envs/pommerman/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Import error! You will not be able to render --> Cannot connect to "None"


In [3]:
batch_size = 2048
epochs = 150
early_stopping = 5
action_space = 6

log_path = './supervised_learning/logs/go_3res_block_elu_tanh/'
model_path = './supervised_learning/model/go_3res_block_elu_tanh/model.h4'

train_data_path    = './dataset/'
train_data_labels  = os.path.join(train_data_path, 'labels.npy')
train_data_reward  = os.path.join(train_data_path, 'reward.npy')
train_data_obs_map = os.path.join(train_data_path, 'obs_map.npy')

if not os.path.isdir(train_data_path):
    os.makedirs(train_data_path)

In [4]:
class Trainer:
    def __init__(self, actions, save_path, log_path, save_best_only=True, seed=0):
        K.clear_session()
        self.log_path = log_path
        self.save_path = save_path
        self.actions = actions
        self.save_best_only = save_best_only
        
        # Create model
        self.model = self.create_model(actions)
        # Load model if exists
        if not os.path.isdir(os.path.dirname(save_path)):
            os.makedirs(os.path.dirname(save_path))            
        if os.path.isfile(self.save_path):
            try:
                print("Trying to load model")
                self.model.load_weights(self.save_path)
                print("Model was loaded successful")
            except:
                print("Model load failed")
        
    def get_res_block(self, input):
        # Res block 1        
        x = Convolution2D(256, 3, padding='same')(input)
        x = BatchNormalization()(x)
        x = Activation('elu')(x)
        x = Convolution2D(256, 3, padding='same')(x)
        x = BatchNormalization()(x)
        x = Add()([input, x])
        x = Activation('elu')(x)
        return x
        
    def create_model(self, actions, input_shape=(11, 11, 18,)):
        inp = Input(input_shape)
        x = Convolution2D(256, 3, padding='same')(inp)
        x = BatchNormalization()(x)
        x = Activation('elu')(x)
        
        # 3 residual blocks
        for i in range(3):
            x = self.get_res_block(x)
        
        # Output block
        # Should be 2 filters
        x = Convolution2D(2, 1, padding='same')(x)
        x = BatchNormalization()(x)   
        x = Activation('elu')(x)
        x = Flatten()(x)  
        
        probs  = Dense(actions, activation='softmax', name='actions')(x)
        reward = Dense(1, activation='tanh', name='reward')(x)
        
        model = Model(inputs = inp, outputs=[probs, reward])
        model.compile(optimizer='adam', loss=['categorical_crossentropy', 'mae'], metrics=['accuracy'])
        return model

    def train(self, obs, actions, rewards, batch_size=16384, epochs=100,
              early_stopping = 10, class_weight=None, initial_epoch=0):
        
        early_stopping = EarlyStopping(monitor='loss', patience=early_stopping)
        checkpoint     = ModelCheckpoint(self.save_path, monitor='loss', save_best_only=self.save_best_only)
        reduce_lr      = ReduceLROnPlateau(monitor='loss', patience=2, factor=0.1)
        logger         = CSVLogger(self.log_path + 'log.csv', append=True)
        tensorboard    = TensorBoard(self.log_path, batch_size=batch_size)
        
        history = self.model.fit(x=obs, y=[actions, rewards], batch_size=batch_size, epochs=epochs, verbose=1,
                       callbacks=[early_stopping, checkpoint, reduce_lr, logger, tensorboard],
                       validation_split=0.15, shuffle=True, class_weight=class_weight, initial_epoch=initial_epoch)

In [5]:
labels       = np.load(train_data_labels)
observations = np.load(train_data_obs_map)
rewards      = np.load(train_data_reward)

In [6]:
labels = to_categorical(labels, num_classes=action_space)

In [7]:
labels.shape, observations.shape, rewards.shape

((553065, 6), (553065, 11, 11, 18), (553065,))

In [8]:
np.sum(labels, axis=0) / np.sum(labels)

array([0.15941164, 0.20397964, 0.190945  , 0.20009764, 0.20339382,
       0.04217226], dtype=float32)

In [9]:
class_weights = compute_class_weight('balanced', np.unique(np.argmax(labels, axis=1)), np.argmax(labels, axis=1))
class_weights

array([1.04551126, 0.81707501, 0.87285166, 0.83292671, 0.81942839,
       3.9520451 ])

In [10]:
trainer = Trainer(action_space, model_path, log_path)

In [11]:
trainer.model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 11, 11, 18)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 11, 11, 256)  41728       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 11, 11, 256)  1024        conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 11, 11, 256)  0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [ ]:
trainer.train(observations, labels, rewards, batch_size=batch_size, 
              epochs=epochs, early_stopping=early_stopping, class_weight=[class_weights, 1])

Train on 470105 samples, validate on 82960 samples
Epoch 1/150
470105/470105 [==============================] - 131s 279us/step - loss: 2.0955 - actions_loss: 1.6011 - reward_loss: 0.4944 - actions_acc: 0.2877 - reward_acc: 0.7519 - val_loss: 2.0442 - val_actions_loss: 1.5383 - val_reward_loss: 0.5059 - val_actions_acc: 0.3420 - val_reward_acc: 0.7480
Epoch 2/150
470105/470105 [==============================] - 127s 271us/step - loss: 1.8446 - actions_loss: 1.3532 - reward_loss: 0.4914 - actions_acc: 0.3999 - reward_acc: 0.7544 - val_loss: 1.9442 - val_actions_loss: 1.4394 - val_reward_loss: 0.5048 - val_actions_acc: 0.3779 - val_reward_acc: 0.7480
Epoch 3/150
470105/470105 [==============================] - 127s 271us/step - loss: 1.5806 - actions_loss: 1.0892 - reward_loss: 0.4914 - actions_acc: 0.5145 - reward_acc: 0.7544 - val_loss: 1.5341 - val_actions_loss: 1.0291 - val_reward_loss: 0.5050 - val_actions_acc: 0.5546 - val_reward_acc: 0.7480
Epoch 4/150
470105/470105 [=============

Epoch 28/150
470105/470105 [==============================] - 127s 271us/step - loss: 1.0768 - actions_loss: 0.5856 - reward_loss: 0.4912 - actions_acc: 0.7333 - reward_acc: 0.7544 - val_loss: 1.3446 - val_actions_loss: 0.8405 - val_reward_loss: 0.5041 - val_actions_acc: 0.6279 - val_reward_acc: 0.7480
Epoch 29/150
470105/470105 [==============================] - 127s 271us/step - loss: 1.0634 - actions_loss: 0.5722 - reward_loss: 0.4912 - actions_acc: 0.7408 - reward_acc: 0.7544 - val_loss: 1.3883 - val_actions_loss: 0.8842 - val_reward_loss: 0.5041 - val_actions_acc: 0.6316 - val_reward_acc: 0.7479
Epoch 30/150
470105/470105 [==============================] - 127s 271us/step - loss: 1.0521 - actions_loss: 0.5609 - reward_loss: 0.4911 - actions_acc: 0.7463 - reward_acc: 0.7544 - val_loss: 1.4147 - val_actions_loss: 0.9107 - val_reward_loss: 0.5041 - val_actions_acc: 0.6233 - val_reward_acc: 0.7480
Epoch 31/150
470105/470105 [==============================] - 127s 271us/step - loss: 1.

In [14]:
print("finish")

finish
